# Advent of Code

## Day 1

Find the top three Elves carrying the most Calories. How many Calories are those Elves carrying in total?

In [1]:
from itertools import groupby

topk = 3

with open("data/day1.txt", "r") as f:
    data = [l.strip() for l in f.readlines()]
    
sum(sorted(sum(map(int, v)) for k, v in groupby(data, key=lambda l: l != '') if k)[-topk:])

212520

## Day 2

What would your total score be if everything goes exactly according to your strategy guide?

In [2]:
with open("data/day2.txt", "r") as f:
    data = [l.strip().split(' ') for l in f.readlines()]

first_col = {k: v for v, k in enumerate('ABC')}
second_col = {k: v for v, k in enumerate('XYZ')}
mapping = list(map(lambda x: (first_col[x[0]], second_col[x[1]]), data))

shape_points = map(lambda x: 1 + x[1], mapping)
outcome_points = map(lambda y : 3 * (y + 1), [v if abs(v) != 2 else -v // 2 for v in map(lambda x: x[1] - x[0], mapping)])
sum(shape_points) + sum(outcome_points)

9759

In [3]:
shape_points = map(lambda x: 1 + ((x[0] + x[1] - 1) % 3), mapping)
outcome_points = map(lambda x: 3 * second_col[x[1]], data)
sum(shape_points) + sum(outcome_points)

12429

## Day 3

Find the item type that appears in both compartments of each rucksack. What is the sum of the priorities of those item types?

In [4]:
import string

with open("data/day3.txt", "r") as f:
    data = [l.strip() for l in f.readlines()]

mapping = {k: v for v, k in enumerate(string.ascii_letters, 1)}

def get_priorities(string):
    split = len(string) // 2
    a, b = string[:split], string[split:]
    return sum(mapping[item] for item in set(a).intersection(b))

priorities = map(get_priorities, data)
sum(priorities)

8139

In [5]:
def get_priorities(triplet):
    a, b, c = triplet
    return sum(mapping[item] for item in set(a).intersection(b).intersection(c))

data_groups = zip(*(iter(data),) * 3)

priorities = map(get_priorities, data_groups)
sum(priorities)

2668

## Day 4

In how many assignment pairs does one range fully contain the other?

In [6]:
with open("data/day4.txt", "r") as f:
    data = [list(map(lambda x: x.split('-'), l.strip().split(','))) for l in f.readlines()]

def fully_overlap(a, b):
    return (int(a[0]) < int(b[0])) ^ (int(a[1]) < int(b[1])) or a[0] == b[0] or a[1] == b[1]
    
sum(fully_overlap(a, b) for a, b in data)

503

In [7]:
def overlap(a, b):
    return (int(a[0]) <= int(b[1])) and (int(a[1]) >= int(b[0]))
    
sum(overlap(a, b) for a, b in data)

827

## Day 5

After the rearrangement procedure completes, what crate ends up on top of each stack?

In [8]:
with open("data/day5.txt", "r") as f:
    stacks = list(reversed([next(f).strip('\n') for _ in range(8)]))
    stacks_cpy = {k+1: [line[4*k+1] for line in stacks if line[4*k+1] != ' '] for k in range(9)}
    for _ in range(2):
        next(f)
    instructions = [dict(zip(*(iter(l.strip().split()),) * 2)) for l in f.readlines()]

for instr in instructions:
    for _ in range(int(instr['move'])):
        stacks_cpy[int(instr['to'])].append(stacks_cpy[int(instr['from'])].pop())
        
''.join(v[-1] for v in stacks_cpy.values())

'RLFNRTNFB'

In [9]:
stacks_cpy = {k+1: [line[4*k+1] for line in stacks if line[4*k+1] != ' '] for k in range(9)}

for instr in instructions:
    pack = list(reversed([stacks_cpy[int(instr['from'])].pop() for _ in range(int(instr['move']))]))
    stacks_cpy[int(instr['to'])].extend(pack)
    
''.join(v[-1] for v in stacks_cpy.values())

'MHQTLJRLB'

## Day 6

How many characters need to be processed before the first start-of-packet marker is detected?

In [10]:
with open("data/day6.txt", "r") as f:
    data = next(f).strip()

[len(set(data[i-4:i])) for i in range(len(data))].index(4)

1909

In [11]:
[len(set(data[i-14:i])) for i in range(len(data))].index(14)

3380

## Day 7

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [12]:
from collections import defaultdict

with open("data/day7.txt", "r") as f:
    data = [line.strip().split() for line in f]
    
system = defaultdict(int)

for line in data:
    if line[0] == '$':
        if line[1] == 'cd':
            if line[2] == '/':
                current = ['.']
            elif line[2] == '..':
                current = current[:-1]
            else:
                current.append(line[2])
    if line[0].isdigit():
        for i, _ in enumerate(current, 1):
            system['/'.join(current[:i])] += int(line[0])
            
sum(v for v in system.values() if v < 100000)

1141028

In [13]:
needed = system['.'] - 40000000
min(v for v in system.values() if v >= needed)

8278005

## Day 8

Consider your map; how many trees are visible from outside the grid?

In [14]:
import numpy as np

data = np.genfromtxt("data/day8.txt", delimiter=1)

up = np.diff(np.maximum.accumulate(data, axis=0), prepend=-1, axis=0)
down = np.flip(np.diff(np.maximum.accumulate(np.flip(data, axis=0), axis=0), prepend=-1, axis=0), axis=0)
left = np.diff(np.maximum.accumulate(data, axis=1), prepend=-1, axis=1)
right = np.flip(np.diff(np.maximum.accumulate(np.flip(data, axis=1), axis=1), prepend=-1, axis=1), axis=1)

np.sum(np.logical_or.reduce([up, down, left, right]))

1816

In [15]:
def vision_(x):
    ahead = np.minimum.accumulate(x[1:] < x[0], axis=0)
    return ahead.sum(axis=0) + (~ahead).any(axis=0)

vision = lambda x: np.array([vision_(x[i:]) for i in range(0, x.shape[0])])

up = vision(data)
down = np.flip(vision(np.flip(data, axis=0)), axis=0)
left = vision(data.T).T
right = np.flip(vision(np.flip(data, axis=1).T).T, axis=1)

np.max(up * down * left * right)

383520